
# Python Async Inference Tutorial - Single Model

This tutorial describes how to run an inference process using `InferModel` (Async) API, which is the recommended option


**Requirements:**

* Run the notebook inside the Python virtual environment: ```source hailo_virtualenv/bin/activate```

When inside the ```virtualenv```, use the command ``jupyter-notebook <tutorial-dir>`` to open a Jupyter server that contains the tutorials (default folder on GitHub: ``hailort/libhailort/bindings/python/platform/hailo_tutorials/notebooks/``).

### HEF
An HEF is Hailo's binary format for neural networks. The HEF files contain:

* Target HW configuration
* Weights
* Metadata for HailoRT (e.g., input/output scaling)



In [ ]:
import numpy as np
from hailo_platform import VDevice

timeout_ms = 1000

# The vdevice is used as a context manager ("with" statement) to ensure it's released on time.
with VDevice() as vdevice:

    # Create an infer model from an HEF:
    infer_model = vdevice.create_infer_model('../hefs/resnet_v1_18.hef')

    # Configure the infer model and create bindings for it
    with infer_model.configure() as configured_infer_model:
        bindings = configured_infer_model.create_bindings()

        # Set input and output buffers
        buffer = np.empty(infer_model.input().shape).astype(np.uint8)
        bindings.input().set_buffer(buffer)

        buffer = np.empty(infer_model.output().shape).astype(np.uint8)
        bindings.output().set_buffer(buffer)

        # Run synchronous inference and access the output buffers
        configured_infer_model.run([bindings], timeout_ms)
        buffer = bindings.output().get_buffer()

        # Run asynchronous inference
        job = configured_infer_model.run_async([bindings])
        job.wait(timeout_ms)